In [11]:
import pandas as pd, requests, pymongo, psycopg2, numpy as np
import pandas.io.sql as sqlio

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

import warnings

warnings.filterwarnings('ignore')

# Crime Data

In [4]:
response = requests.get("https://data.lacity.org/api/views/63jg-8b9z/rows.json?accessType=DOWNLOAD")


In [5]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = myclient["DAP_souvik"]
mycol = mydb["crime"]

In [6]:
all_data =response.json()
cols = [col_name["name"] for col_name in all_data["meta"]["view"]["columns"]]

In [7]:
# all_data =response.json()
# making key-value pairs
values=list(map(list, zip(*[data for data in all_data["data"]])))


In [8]:
keyvalues =[dict(zip(cols, item)) for item in zip(*values)]

# Inserting Data to MongoDB

In [1]:
many_ids = mycol.insert_many(keyvalues).inserted_ids


# Fetching From MongoDB

In [2]:
# fetching data from mongodb
dff=[]
for i in mycol.find():
    dff.append(i)
    
df = pd.DataFrame(dff)


In [9]:
dataframe= pd.DataFrame(keyvalues)

In [14]:
dataframe.head()

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,row-a3qu_3mjr~vtsq,00000000-0000-0000-D569-8211F0C7D24E,0,1561495080,None,1561495080,None,{ },001307355,2010-02-20T00:00:00,2010-02-20T00:00:00,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,48,M,H,501,SINGLE FAMILY DWELLING,None,None,AA,Adult Arrest,900,None,None,None,300 E GAGE AV,None,33.9825,-118.2695
1,row-pm9b-z3f4~zmie,00000000-0000-0000-4559-BEC3CF9E294C,0,1561495080,None,1561495080,None,{ },011401303,2010-09-13T00:00:00,2010-09-12T00:00:00,0045,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,0,M,W,101,STREET,None,None,IC,Invest Cont,740,None,None,None,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,row-9res~fyki-9yjh,00000000-0000-0000-1505-911AD4C50AD3,0,1561495080,None,1561495080,None,{ },070309629,2010-08-09T00:00:00,2010-08-09T00:00:00,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,0344,0,M,H,103,ALLEY,None,None,IC,Invest Cont,946,None,None,None,1300 E 21ST ST,None,34.0224,-118.2524
3,row-czjz_3usw~mtms,00000000-0000-0000-87AA-0B80C4FA4C78,0,1561495080,None,1561495080,None,{ },090631215,2010-01-05T00:00:00,2010-01-05T00:00:00,0150,06,Hollywood,0646,2,900,VIOLATION OF COURT ORDER,1100 0400 1402,47,F,W,101,STREET,102,HAND GUN,IC,Invest Cont,900,998,None,None,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295
4,row-ha93.zbff-entb,00000000-0000-0000-6576-3B12B1DE6303,0,1561495080,None,1561495080,None,{ },100100501,2010-01-03T00:00:00,2010-01-02T00:00:00,2100,01,Central,0176,1,122,"RAPE, ATTEMPTED",0400,47,F,H,103,ALLEY,400,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,122,None,None,None,8TH ST,SAN PEDRO ST,34.0387,-118.2488


In [59]:
dataframe.columns

Index(['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at',
       'updated_meta', 'meta', 'DR_NO', 'Date Rptd', 'DATE OCC', 'TIME OCC',
       'AREA ', 'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd',
       'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent',
       'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status',
       'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4',
       'LOCATION', 'Cross Street', 'LAT', 'LON'],
      dtype='object')

In [60]:
dataframe.drop(['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at',
       'updated_meta', 'meta', 'DR_NO', 'Date Rptd', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4',
       'LOCATION', 'Cross Street', 'LAT', 'LON'],axis=1,inplace=True)

In [61]:
dataframe.drop(['Mocodes'],axis=1,inplace=True)

In [66]:
dataframe['Vict Age'].isna().sum()
dataframe['Vict Age'].fillna(0,inplace=True)

In [67]:
dataframe['Vict Age']=pd.to_numeric(dataframe['Vict Age'])
n = int(sum(dataframe['Vict Age'])/len(dataframe['Vict Age']))
dataframe['Vict Age'].replace(0,35,inplace=True)

In [69]:
limit=[]
for i in dataframe['Vict Age']:
    if i<19:
        limit.append('0-18')
    elif i<31:
        limit.append('19-30')
    elif i<46:
        limit.append('31-45')
    elif i<61:
        limit.append('46-60')
    else:
        limit.append('60+')
dataframe['Age group']=limit

In [70]:
dataframe['Age group'].value_counts()

31-45    908143
19-30    494893
46-60    370001
60+      170076
0-18     117835
Name: Age group, dtype: int64

In [71]:
df1=pd.get_dummies(df['Age group'])

In [72]:
df1.head()

,0-18,19-30,31-45,46-60,60+
0,0,0,0,1,0
1,0,0,1,0,0
2,0,0,1,0,0
3,0,0,0,1,0
4,0,0,0,1,0


In [73]:
dataframe['0-18']=df1['0-18']
dataframe['19-30']=df1['19-30']
dataframe['31-45']=df1['31-45']
dataframe['46=60']=df1['46-60']
dataframe['60+']=df1['60+']

In [74]:
dataframe.head()

,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Age group,0-18,19-30,31-45,46=60,60+
0,2010-02-20T00:00:00,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,48,M,H,501,SINGLE FAMILY DWELLING,None,None,AA,Adult Arrest,46-60,0.0,0.0,0.0,1.0,0.0
1,2010-09-12T00:00:00,0045,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",35,M,W,101,STREET,None,None,IC,Invest Cont,31-45,0.0,0.0,1.0,0.0,0.0
2,2010-08-09T00:00:00,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,35,M,H,103,ALLEY,None,None,IC,Invest Cont,31-45,0.0,0.0,1.0,0.0,0.0
3,2010-01-05T00:00:00,0150,06,Hollywood,0646,2,900,VIOLATION OF COURT ORDER,47,F,W,101,STREET,102,HAND GUN,IC,Invest Cont,46-60,0.0,0.0,0.0,1.0,0.0
4,2010-01-02T00:00:00,2100,01,Central,0176,1,122,"RAPE, ATTEMPTED",47,F,H,103,ALLEY,400,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,46-60,0.0,0.0,0.0,1.0,0.0


In [75]:
# converting required columns to numeric type
for i in  dataframe.columns:
    try:
        dataframe[i]=pd.to_numeric(dataframe[i])
    except:
        pass

In [76]:
dataframe['DATE OCC']=pd.to_datetime(dataframe['DATE OCC'])

In [77]:
dataframe.dtypes

DATE OCC          datetime64[ns]
TIME OCC                   int64
AREA                       int64
AREA NAME                 object
Rpt Dist No                int64
Part 1-2                   int64
Crm Cd                     int64
Crm Cd Desc               object
Vict Age                   int64
Vict Sex                  object
Vict Descent              object
Premis Cd                float64
Premis Desc               object
Weapon Used Cd           float64
Weapon Desc               object
Status                    object
Status Desc               object
Age group                 object
0-18                     float64
19-30                    float64
31-45                    float64
46=60                    float64
60+                      float64
dtype: object

In [78]:
dataframe=dataframe[dataframe['Status'].isin(['IC','AO','AA','JA'])]

In [79]:
dataframe['Status Desc'].value_counts()

Invest Cont     1587396
Adult Other      244103
Adult Arrest     210365
Juv Arrest        14200
Name: Status Desc, dtype: int64

In [80]:
dataframe['Weapon Used Cd'].value_counts()

400.0    417620
500.0     58541
511.0     56941
102.0     34289
109.0     13387
200.0     13076
207.0      9733
106.0      8838
307.0      7159
101.0      5182
212.0      5150
308.0      4625
306.0      4605
512.0      4346
304.0      4325
204.0      4313
302.0      4016
205.0      3799
201.0      3493
113.0      3416
114.0      2882
312.0      2713
301.0      2409
218.0      2228
515.0      1809
311.0      1666
219.0      1606
223.0      1429
215.0      1325
216.0      1143
104.0      1109
107.0      1005
310.0       973
103.0       859
305.0       851
513.0       676
506.0       627
309.0       596
206.0       552
501.0       515
221.0       479
503.0       473
303.0       423
514.0       360
211.0       343
112.0       311
510.0       300
504.0       296
210.0       245
217.0       204
208.0       188
108.0       175
115.0       165
505.0       161
214.0       161
122.0       160
203.0       136
105.0       132
507.0       129
111.0        98
509.0        91
110.0        90
213.0   

In [16]:
df4= pd.get_dummies(dataframe['Status'])
df4.head()

,13,AA,AO,CC,IC,JA,JO,TH
0,0,1,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0
2,0,0,0,0,1,0,0,0
3,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,0,0


In [17]:
dataframe['investigation']=df4['IC']
dataframe['adult_other']=df4['AO']
dataframe['adult_arrest']=df4['AA']
dataframe['juvenile_arrest']=df4['JA']

In [89]:
df1= pd.get_dummies(dataframe['Vict Sex'])

In [90]:
df1.head()

,-,F,H,M,N,X
0,0,0,0,1,0,0
1,0,0,0,1,0,0
2,0,0,0,1,0,0
3,0,1,0,0,0,0
4,0,1,0,0,0,0


In [19]:
df1= pd.get_dummies(dataframe['Vict Descent'])

In [20]:
df1.head()

,-,A,B,C,D,F,G,H,I,J,K,L,O,P,S,U,V,W,X,Z
0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
dataframe['VD_B']=df1['B']
dataframe['VD_W']=df1['W']
dataframe['VD_A']=df1['A']
dataframe['VD_H']=df1['H']
dataframe['VD_O']=df1['O']


In [22]:
df1= pd.get_dummies(dataframe['Vict Sex'])
dataframe['Male']=df1['M']
dataframe['Female']=df1['F']

In [23]:
# weapon used or not
dataframe['weapon used'] = list(map(lambda x: 0 if x==0 else 1, dataframe['Weapon Desc']))

In [24]:
dataframe['weapon used'].value_counts()

1    2060948
Name: weapon used, dtype: int64

In [3]:
dataframe['hr']=dataframe['TIME OCC']/100
dataframe['hr'].astype(int)


In [7]:
dataframe['min']=dataframe['TIME OCC']%100
dataframe['min'].astype(int)


In [98]:
dataframe.head()

,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,Age group,0-18,19-30,31-45,46=60,60+,investigation,adult_other,adult_arrest,juvenile_arrest,VD_B,VD_W,VD_A,VD_H,VD_O,Male,Female,weapon used,hr,min
0,2010-02-20,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,48,M,H,501.0,SINGLE FAMILY DWELLING,NaN,None,AA,Adult Arrest,46-60,0.0,0.0,0.0,1.0,0.0,0,0,1,0,0,0,0,1,0,1,0,1,13.50,50
1,2010-09-12,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",35,M,W,101.0,STREET,NaN,None,IC,Invest Cont,31-45,0.0,0.0,1.0,0.0,0.0,1,0,0,0,0,1,0,0,0,1,0,1,0.45,45
2,2010-08-09,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,35,M,H,103.0,ALLEY,NaN,None,IC,Invest Cont,31-45,0.0,0.0,1.0,0.0,0.0,1,0,0,0,0,0,0,1,0,1,0,1,15.15,15
3,2010-01-05,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,47,F,W,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,46-60,0.0,0.0,0.0,1.0,0.0,1,0,0,0,0,1,0,0,0,0,1,1,1.50,50
4,2010-01-02,2100,1,Central,176,1,122,"RAPE, ATTEMPTED",47,F,H,103.0,ALLEY,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,46-60,0.0,0.0,0.0,1.0,0.0,1,0,0,0,0,0,0,1,0,0,1,1,21.00,0


# Creating crimedata table for individual data analysis

In [4]:
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "127.0.0.1",
        port = "5432",
        database = "dap")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    qr = '''CREATE TABLE crimedata(



    
    date_occured timestamp , 
    time_count integer,
    area integer,
    area_name integer,
    report_no integer,
    part integer,
    crime_description varchar(1000),
    victim_age float,
    victim_sex float,
    victim_description varchar(500),
    premise_code integer,
    primise_descendent varchar(1000),
    weapon_used float,
    weapon_desc varchar(500),
    status varchar(100),
    status_desc varchar(100),
    age_group integer,
    year0_18 float,
    year19_30 float,
    year31_45 float,
    year46_60 float,
    year60abv float,
    investigation float,
    adult_others float,
    adult_arrest float,
    juvenile_arrest float,
    victDesc_black float,
    victDesc_white float,
    victDesc_asian float,
    victDesc_hispanic float,
    victDesc_others float,
    male float,
    female float,
    weapon_used float,
    minute float,
    night float,
    morning float,
    afternoon float,
    evening float,
    hour integer,
    timeofday varchar(1000)
    );
    '''
    dbCursor.execute(qr)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL step 2", dbError)
finally:
    if dbConnection in locals(): 
        dbConnection.close()


# Inserting Data into crimedata table for individual analysis

In [6]:
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "127.0.0.1",
        port = "5432",
        database = "dap")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()    
    print('done')

    tpls = [tuple(x) for x in dataframe.to_numpy()]
    sql32 = '''INSERT INTO crimedata( 
    
    date_occured, 
    time_count,
    area,
    area_name,
    report_no,
    part,
    crime_description,
    victim_age,
    victim_sex,
    victim_description,
    premise_code,
    primise_descendent,
    weapon_used,
    weapon_desc,
    status,
    status_desc,
    age_group,
    year0_18,
    year19_30,
    year31_45,
    year46_60,
    year60abv,
    investigation,
    adult_others,
    adult_arrest,
    juvenile_arrest,
    victDesc_black,
    victDesc_white,
    victDesc_asian,
    victDesc_hispanic,
    victDesc_others,
    male,
    female,
    weapon_used,
    minute,
    night,
    morning,
    afternoon,
    evening,
    hour,
    timeofday 
       VALUES('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}',
       '{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}');'''

    for num,row in enumerate(tpls):
        # print("Data inserted using execute_many() successfully...")
        rw = sql32.format(*row)
        dbCursor.execute(rw)

    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
    print(num,row)
    print(rw)
finally:
    if dbConnection in locals(): 
        dbConnection.close()

done


# Processing Data for Final joining and combined analysis

In [99]:
ddf=dataframe.groupby('DATE OCC', as_index=False).agg({'Status Desc':'count','hr': np.mean,'AREA NAME':pd.Series.mode,'AREA ':pd.Series.mode,
                                              'Crm Cd':pd.Series.mode, 'Crm Cd Desc':pd.Series.mode,'Vict Age':np.median, 
                                                'Premis Desc':pd.Series.mode, 'Premis Cd':pd.Series.mode,
                                               'weapon used':sum, 'Male':sum,'Female':sum,'VD_B':sum,'VD_W':sum,'VD_A':sum,'VD_H':sum,'VD_O':sum,'weapon used':sum,'investigation':sum,'adult_other':sum,'adult_arrest':sum, 'juvenile_arrest':sum,'0-18':sum,'19-30':sum,'31-45':sum,'46=60':sum, '60+':sum,})

In [43]:
ddf['hr']=ddf['hr'].astype(int)

ddf.head(10)

,DATE OCC,Status Desc,hr,AREA NAME,AREA,Crm Cd,Crm Cd Desc,Vict Age,Premis Desc,Premis Cd,weapon used,Male,Female,VD_B,VD_W,VD_A,VD_H,VD_O,investigation,adult_other,adult_arrest,juvenile_arrest,0-18,19-30,31-45,46=60,60+
0,2010-01-01,2304,8,Mission,19,354,THEFT OF IDENTITY,35.0,SINGLE FAMILY DWELLING,501.0,2304,964.0,1259.0,361.0,422.0,43,1156.0,215,1915.0,247.0,132,10,536.0,470.0,589.0,406.0,303.0
1,2010-01-02,535,14,Mission,19,624,BATTERY - SIMPLE ASSAULT,35.0,STREET,101.0,535,255.0,222.0,83.0,131.0,11,191.0,49,441.0,45.0,46,3,18.0,134.0,248.0,101.0,34.0
2,2010-01-03,538,13,77th Street,12,624,BATTERY - SIMPLE ASSAULT,35.0,STREET,101.0,538,247.0,231.0,84.0,142.0,6,201.0,42,448.0,35.0,52,3,24.0,137.0,257.0,91.0,29.0
3,2010-01-04,557,13,77th Street,12,"[330, 624]","[BATTERY - SIMPLE ASSAULT, BURGLARY FROM VEHICLE]",35.0,STREET,101.0,557,245.0,250.0,87.0,138.0,13,196.0,49,477.0,39.0,39,2,34.0,153.0,222.0,102.0,46.0
4,2010-01-05,547,13,Southwest,3,310,BURGLARY,35.0,SINGLE FAMILY DWELLING,501.0,547,241.0,244.0,92.0,158.0,13,169.0,46,460.0,47.0,38,2,35.0,127.0,239.0,106.0,40.0
5,2010-01-06,532,14,"[Southwest, West Valley]","[3, 10]","[310, 330]","[BURGLARY, BURGLARY FROM VEHICLE]",35.0,"[SINGLE FAMILY DWELLING, STREET]","[101.0, 501.0]",532,270.0,215.0,85.0,156.0,5,171.0,49,430.0,51.0,50,1,28.0,126.0,246.0,91.0,41.0
6,2010-01-07,550,13,Southwest,3,310,BURGLARY,35.0,SINGLE FAMILY DWELLING,501.0,550,253.0,243.0,89.0,136.0,8,205.0,49,463.0,40.0,44,3,37.0,139.0,249.0,85.0,40.0
7,2010-01-08,605,14,Southwest,3,624,BATTERY - SIMPLE ASSAULT,35.0,STREET,101.0,605,304.0,234.0,114.0,147.0,11,210.0,49,502.0,50.0,50,3,39.0,133.0,278.0,105.0,50.0
8,2010-01-09,631,13,Southwest,3,624,BATTERY - SIMPLE ASSAULT,35.0,STREET,101.0,631,305.0,265.0,82.0,169.0,19,226.0,60,526.0,43.0,59,3,46.0,192.0,262.0,97.0,34.0
9,2010-01-10,617,12,77th Street,12,624,BATTERY - SIMPLE ASSAULT,35.0,STREET,101.0,617,302.0,256.0,94.0,162.0,12,227.0,58,523.0,51.0,42,1,54.0,180.0,245.0,104.0,34.0


In [44]:
ddf.rename(columns = {'Status Desc':'Crime_count'}, inplace = True)

In [45]:
ddf.drop(['AREA NAME','Crm Cd','Premis Cd'],axis=1,inplace=True)

In [46]:
ddf.head()

,DATE OCC,Crime_count,hr,AREA,Crm Cd Desc,Vict Age,Premis Desc,weapon used,Male,Female,VD_B,VD_W,VD_A,VD_H,VD_O,investigation,adult_other,adult_arrest,juvenile_arrest,0-18,19-30,31-45,46=60,60+
0,2010-01-01,2304,8,19,THEFT OF IDENTITY,35.0,SINGLE FAMILY DWELLING,2304,964.0,1259.0,361.0,422.0,43,1156.0,215,1915.0,247.0,132,10,536.0,470.0,589.0,406.0,303.0
1,2010-01-02,535,14,19,BATTERY - SIMPLE ASSAULT,35.0,STREET,535,255.0,222.0,83.0,131.0,11,191.0,49,441.0,45.0,46,3,18.0,134.0,248.0,101.0,34.0
2,2010-01-03,538,13,12,BATTERY - SIMPLE ASSAULT,35.0,STREET,538,247.0,231.0,84.0,142.0,6,201.0,42,448.0,35.0,52,3,24.0,137.0,257.0,91.0,29.0
3,2010-01-04,557,13,12,"[BATTERY - SIMPLE ASSAULT, BURGLARY FROM VEHICLE]",35.0,STREET,557,245.0,250.0,87.0,138.0,13,196.0,49,477.0,39.0,39,2,34.0,153.0,222.0,102.0,46.0
4,2010-01-05,547,13,3,BURGLARY,35.0,SINGLE FAMILY DWELLING,547,241.0,244.0,92.0,158.0,13,169.0,46,460.0,47.0,38,2,35.0,127.0,239.0,106.0,40.0


In [19]:
import psycopg2
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "127.0.0.1",
        port = "5432",
        database = "postgres")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    dbCursor.execute('CREATE DATABASE DAP;')
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL step 1", dbError)
finally:
    if dbConnection in locals(): 
        dbConnection.close()


In [172]:
# ddf.columns

In [251]:
ddf=pd.DataFrame(ddf)
ddf.head()

,DATE OCC,Crime_count,hr,AREA,Crm Cd Desc,Vict Age,Premis Desc,weapon used,Male,Female,VD_B,VD_W,VD_A,VD_H,VD_O,investigation,adult_other,adult_arrest,juvenile_arrest,0-18,19-30,31-45,46=60,60+
0,2010-01-01,2304,8,19,THEFT OF IDENTITY,35.0,SINGLE FAMILY DWELLING,1.0,0.418403,0.546441,0.156684,0.183160,0.018663,0.501736,0.093316,0.831163,0.107205,0.057292,0.004340,536.0,470.0,589.0,406.0,303.0
1,2010-01-02,535,14,19,BATTERY - SIMPLE ASSAULT,35.0,STREET,1.0,0.476636,0.414953,0.155140,0.244860,0.020561,0.357009,0.091589,0.824299,0.084112,0.085981,0.005607,18.0,134.0,248.0,101.0,34.0
2,2010-01-03,538,13,12,BATTERY - SIMPLE ASSAULT,35.0,STREET,1.0,0.459108,0.429368,0.156134,0.263941,0.011152,0.373606,0.078067,0.832714,0.065056,0.096654,0.005576,24.0,137.0,257.0,91.0,29.0
3,2010-01-04,557,13,12,['BATTERY - SIMPLE ASSAULT' 'BURGLARY FROM VEH...,35.0,STREET,1.0,0.439856,0.448833,0.156194,0.247756,0.023339,0.351885,0.087971,0.856373,0.070018,0.070018,0.003591,34.0,153.0,222.0,102.0,46.0
4,2010-01-05,547,13,3,BURGLARY,35.0,SINGLE FAMILY DWELLING,1.0,0.440585,0.446069,0.168190,0.288848,0.023766,0.308958,0.084095,0.840951,0.085923,0.069470,0.003656,35.0,127.0,239.0,106.0,40.0


In [252]:
ddf.columns


Index(['DATE OCC', 'Crime_count', 'hr', 'AREA ', 'Crm Cd Desc', 'Vict Age',
       'Premis Desc', 'weapon used', 'Male', 'Female', 'VD_B', 'VD_W', 'VD_A',
       'VD_H', 'VD_O', 'investigation', 'adult_other', 'adult_arrest',
       'juvenile_arrest', '0-18', '19-30', '31-45', '46=60', '60+'],
      dtype='object')

In [253]:
ddf['Crm Cd Desc']=ddf['Crm Cd Desc'].str.replace("\'","")
ddf['Crm Cd Desc']=ddf['Crm Cd Desc'].str.replace("[","")
ddf['Crm Cd Desc']=ddf['Crm Cd Desc'].str.replace("]","")

ddf['Premis Desc']=ddf['Crm Cd Desc'].str.replace("\'","")
ddf['Premis Desc']=ddf['Crm Cd Desc'].str.replace("[","")
ddf['Premis Desc']=ddf['Crm Cd Desc'].str.replace("]","")


ddf['AREA ']=ddf['Crm Cd Desc'].str.replace("\'","")
ddf['AREA ']=ddf['Crm Cd Desc'].str.replace("[","")
ddf['AREA ']=ddf['Crm Cd Desc'].str.replace("]","")

<ipython-input-253-c73b78b97a1c>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ddf['Crm Cd Desc']=ddf['Crm Cd Desc'].str.replace("[","")
<ipython-input-253-c73b78b97a1c>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ddf['Crm Cd Desc']=ddf['Crm Cd Desc'].str.replace("]","")
<ipython-input-253-c73b78b97a1c>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  ddf['Premis Desc']=ddf['Crm Cd Desc'].str.replace("[","")
<ipython-input-253-c73b78b97a1c>:7: FutureWarning: The default value of regex will change from True to False 

# Creating crimedataFinal table

In [254]:
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "127.0.0.1",
        port = "5432",
        database = "dap")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()
    qr = '''CREATE TABLE crimedataFinal(



    
    crdate_occured timestamp , 
    crdrime_count integer,
    crhour integer,
    crarea varchar(500),
    crcrime_description varchar(1000),
    crvictim_age float,
    crprimise_descendent varchar(1000),
    crweapon_used float,
    crmale float,
    crfemale float,
    crvictDesc_black float,
    crvictDesc_white float,
    crvictDesc_asian float,
    crvictDesc_hispanic float,
    crvictDesc_others float,
    crinvestigation float,
    cradult_others float,
    cradult_arrest float,
    crjuvenile_arrest float,
    cryear0_18 float,
    cryear19_30 float,
    cryear31_45 float,
    cryear46_60 float,
    cryear60abv float
    
             );
    '''
    dbCursor.execute(qr)
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error while connecting to PostgreSQL step 2", dbError)
finally:
    if dbConnection in locals(): 
        dbConnection.close()

In [255]:
ddf.shape

(4353, 24)

In [256]:
tpls = [tuple(x) for x in ddf.to_numpy()]

# Inserting Data into crimedataFinal table

In [257]:
try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "127.0.0.1",
        port = "5432",
        database = "dap")
    dbConnection.set_isolation_level(0) # AUTOCOMMIT
    dbCursor = dbConnection.cursor()    
    print('done')

#     tpls = [tuple(x) for x in df.to_numpy()]
    sql32 = '''INSERT INTO crimedataFinal( crdate_occured, 
    crdrime_count,
    crhour,
    crarea,
    crcrime_description,
    crvictim_age,
    crprimise_descendent,
    crweapon_used,
    crmale,
    crfemale,
    crvictDesc_black,
    crvictDesc_white,
    crvictDesc_asian,
    crvictDesc_hispanic,
    crvictDesc_others,
    crinvestigation,
    cradult_others,
    cradult_arrest,
    crjuvenile_arrest,
    cryear0_18,
    cryear19_30,
    cryear31_45,
    cryear46_60,
    cryear60abv) 
       VALUES('{}','{}','{}','{}','{}','{}','{}','{}','{}','{}',
       '{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}','{}');'''

    for num,row in enumerate(tpls):
        # print("Data inserted using execute_many() successfully...")
        rw = sql32.format(*row)
        dbCursor.execute(rw)

    dbConnection.commit()
    dbCursor.close()
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
    print(num,row)
    print(rw)
finally:
    if dbConnection in locals(): 
        dbConnection.close()

done


# Fetching from crimedataFinal

In [51]:
sql = """
    SELECT
      *
    FROM
      crimedataFinal
    ;"""

try:
    dbConnection = psycopg2.connect(
        user = "dap",
        password = "dap",
        host = "127.0.0.1",
        port = "5432",
        database = "dap"
    )
    sqlData = sqlio.read_sql_query(sql, dbConnection)
except (Exception , psycopg2.Error) as dbError :
    print ("Error:", dbError)
finally:
    if dbConnection in locals(): 
        dbConnection.close()